<a href="https://colab.research.google.com/github/harirm85/Hari_ML_workshop/blob/main/EDA2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

pd.options.display.float_format = '{:,.2f}'.format

In [3]:
# Connect to Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Copy the Jason file and set security
!mkdir -p ~/.kaggle
!cp '/content/drive/MyDrive/Kaggle/kaggle.json' ~/.kaggle/kaggle.json
!chmod 600 ~/.kaggle/kaggle.json

#Download data
!kaggle competitions download -c home-credit-default-risk

import zipfile

# Specify the path to the downloaded zip file
zip_path = "/content/home-credit-default-risk.zip"

# Specify the directory to extract the files to
extract_path = "/content/home-credit-default-risk"

# Extract the zip file
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

Mounted at /content/drive
 99% 681M/688M [00:07<00:00, 47.1MB/s]
100% 688M/688M [00:08<00:00, 90.1MB/s]


In [4]:
app_data = pd.read_csv('/content/home-credit-default-risk/application_train.csv')
prev_data = pd.read_csv('/content/home-credit-default-risk/previous_application.csv')

print(app_data.shape)
print(prev_data.shape)

(307511, 122)
(1670214, 37)


In [5]:
prev_data.info(verbose=True, show_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1670214 entries, 0 to 1670213
Data columns (total 37 columns):
 #   Column                       Non-Null Count    Dtype  
---  ------                       --------------    -----  
 0   SK_ID_PREV                   1670214 non-null  int64  
 1   SK_ID_CURR                   1670214 non-null  int64  
 2   NAME_CONTRACT_TYPE           1670214 non-null  object 
 3   AMT_ANNUITY                  1297979 non-null  float64
 4   AMT_APPLICATION              1670214 non-null  float64
 5   AMT_CREDIT                   1670213 non-null  float64
 6   AMT_DOWN_PAYMENT             774370 non-null   float64
 7   AMT_GOODS_PRICE              1284699 non-null  float64
 8   WEEKDAY_APPR_PROCESS_START   1670214 non-null  object 
 9   HOUR_APPR_PROCESS_START      1670214 non-null  int64  
 10  FLAG_LAST_APPL_PER_CONTRACT  1670214 non-null  object 
 11  NFLAG_LAST_APPL_IN_DAY       1670214 non-null  int64  
 12  RATE_DOWN_PAYMENT            774370 non-nu

In [6]:
prev_data['RATE_INTEREST_PRIMARY'].value_counts()

0.19    1218
0.14     951
0.16     821
0.19     681
0.20     573
        ... 
0.18       1
0.69       1
0.26       1
0.74       1
0.65       1
Name: RATE_INTEREST_PRIMARY, Length: 148, dtype: int64

In [15]:
prev_sample = prev_data[prev_data['SK_ID_CURR']==108129]

prev_sample.iloc[:,10:]

,FLAG_LAST_APPL_PER_CONTRACT,NFLAG_LAST_APPL_IN_DAY,RATE_DOWN_PAYMENT,RATE_INTEREST_PRIMARY,RATE_INTEREST_PRIVILEGED,NAME_CASH_LOAN_PURPOSE,NAME_CONTRACT_STATUS,DAYS_DECISION,NAME_PAYMENT_TYPE,CODE_REJECT_REASON,...,NAME_SELLER_INDUSTRY,CNT_PAYMENT,NAME_YIELD_GROUP,PRODUCT_COMBINATION,DAYS_FIRST_DRAWING,DAYS_FIRST_DUE,DAYS_LAST_DUE_1ST_VERSION,DAYS_LAST_DUE,DAYS_TERMINATION,NFLAG_INSURED_ON_APPROVAL
1,Y,1,NaN,NaN,NaN,XNA,Approved,-164,XNA,XAP,...,XNA,36.00,low_action,Cash X-Sell: low,"365,243.00",-134.00,916.00,"365,243.00","365,243.00",1.00
335841,Y,1,NaN,NaN,NaN,XNA,Approved,-515,XNA,XAP,...,XNA,36.00,low_normal,Cash X-Sell: low,"365,243.00",-485.00,565.00,-155.00,-147.00,1.00
588441,Y,1,0.56,NaN,NaN,XAP,Approved,-619,Cash through the bank,XAP,...,Consumer electronics,6.00,high,POS household with interest,"365,243.00",-588.00,-438.00,-588.00,-580.00,0.00
617224,Y,1,0.00,NaN,NaN,XAP,Approved,-1208,Cash through the bank,XAP,...,Furniture,12.00,low_normal,POS industry with interest,"365,243.00","-1,176.00",-846.00,-846.00,-840.00,0.00
692217,Y,1,NaN,NaN,NaN,XAP,Approved,-957,XNA,XAP,...,XNA,0.00,XNA,Card X-Sell,-713.00,-673.00,"365,243.00",-461.00,-61.00,0.00
1380202,Y,1,0.00,NaN,NaN,XAP,Approved,-819,Cash through the bank,XAP,...,Furniture,4.00,low_action,POS industry with interest,"365,243.00",-783.00,-693.00,-753.00,-748.00,0.00


In [22]:
Summary_data = prev_data.groupby('SK_ID_CURR', as_index=False).agg(
    PRV_CNT = ("SK_ID_PREV","count")
    , PRV_CL_CNT = ("NAME_CONTRACT_TYPE", lambda x: sum(x=='Cash loans'))
    , PRV_CONL_CNT = ("NAME_CONTRACT_TYPE", lambda x: sum(x=='Consumer loans'))
    , PRV_RL_CNT = ("NAME_CONTRACT_TYPE", lambda x: sum(x=='Revolving loans'))
    , PRV_ANNU_MAX = ("AMT_ANNUITY", "max")
    , PRV_ANNU_MIN = ("AMT_ANNUITY", lambda x:min(x[x > 0]) if len(x[x > 0]) > 0 else None)
    , PRV_AMT_APP_MAX = ("AMT_APPLICATION", "max")
    , PRV_AMT_APP_MIN = ("AMT_APPLICATION", lambda x: min(x[x > 0]) if len(x[x > 0]) > 0 else None)
    , PRV_AMT_CRD_MAX = ("AMT_CREDIT", "max")
    , PRV_AMT_CRD_MIN = ("AMT_CREDIT", lambda x:min(x[x > 0]) if len(x[x > 0]) > 0 else None)
    , PRV_AMT_DPAY_MAX = ("AMT_DOWN_PAYMENT", "max")
    , PRV_AMT_DPAY_MIN = ("AMT_DOWN_PAYMENT", lambda x:min(x[x > 0]) if len(x[x > 0]) > 0 else None)
    , PRV_AMT_GPRC_MAX = ("AMT_GOODS_PRICE", "max")
    , PRV_AMT_GPRC_MIN = ("AMT_GOODS_PRICE", lambda x: min(x[x > 0]) if len(x[x > 0]) > 0 else None)
    , PRV_DAY_APPRV = ("WEEKDAY_APPR_PROCESS_START", "nunique")
    , PRV_TIME_APPRV = ("HOUR_APPR_PROCESS_START", "nunique")

    , PRV_RT_DPAY_MAX = ("RATE_DOWN_PAYMENT", "max")
    , PRV_RT_DPAY_MIN = ("RATE_DOWN_PAYMENT", "min")
    , PRV_RT_INT_MAX = ("RATE_INTEREST_PRIMARY", "max")
    , PRV_RT_INT_MIN = ("RATE_INTEREST_PRIMARY", "min")
    , PRV_RT_INT_P_MAX = ("RATE_INTEREST_PRIVILEGED", "max")
    , PRV_RT_INT_P_MIN = ("RATE_INTEREST_PRIVILEGED", "min")
    , PRV_CL_PUR = ("NAME_CASH_LOAN_PURPOSE", "nunique")
    , PRV_CCL_CNT = ("NAME_CONTRACT_STATUS", lambda x: sum(x=='Canceled'))
    , PRV_RFS_CNT = ("NAME_CONTRACT_STATUS", lambda x: sum(x=='Refused'))
    , PRV_UNUS_CNT = ("NAME_CONTRACT_STATUS", lambda x: sum(x=='Unused offer'))
    , PRV_DAYS_DES_MAX = ("DAYS_DECISION", "max")
    , PRV_DAYS_DES_MIN = ("DAYS_DECISION", "min")
    , PRV_PAY_TYP = ("NAME_PAYMENT_TYPE", "nunique")
    , PRV_PAY_TYP_OTHR = ("NAME_PAYMENT_TYPE", lambda x: sum(x.isin(['Cash through the bank','XNA']) == False))
    , PRV_REJ_RSN = ("CODE_REJECT_REASON", lambda x: sum(x.isin(['XAP','XNA']) == False))
    , PRV_TYP_OTHR = ("NAME_TYPE_SUITE", lambda x: sum(x.isin(['Unaccompanied','Family','Spouse, partner']) == False))
    , PRV_PAY_CSH = ("NAME_PORTFOLIO", lambda x: sum(x=='Cash'))
    , PRV_PRD_TYP = ("NAME_PRODUCT_TYPE", lambda x: sum(x=='walk-in'))
    , PRV_CHNL_TYP = ("CHANNEL_TYPE",  lambda x: sum(x=='Regional / Local'))
    , PRV_SLR_AREA = ("SELLERPLACE_AREA", lambda x: min(x[x > 0]) if len(x[x > 0]) > 0 else None)
    , PRV_SLR_TYP = ("NAME_SELLER_INDUSTRY",  lambda x: sum(x.isin(['Jewelry','Tourism','Furniture','Clothing','MLM partners']) == True))
    , PRV_PYMT_MAX = ("CNT_PAYMENT", "max")
    , PRV_PYMT_MIN = ("CNT_PAYMENT", lambda x: min(x[x > 0]) if len(x[x > 0]) > 0 else None)
    , PRV_YILD_TYP = ("NAME_YIELD_GROUP", lambda x: sum(x=='high'))
    , PRV_PRD_CSH = ("PRODUCT_COMBINATION", lambda x: sum(x=='Cash'))

    , PRV_DAYS_DRWN_MIN = ("DAYS_FIRST_DRAWING", lambda x: min(x[x < 0]) if len(x[x < 0]) > 0 else None)
    , PRV_DAYS_DRWN_MAX = ("DAYS_FIRST_DRAWING", lambda x: max(x[x < 0]) if len(x[x < 0]) > 0 else None)
    , PRV_FRST_DUE_MIN = ("DAYS_FIRST_DUE", lambda x: min(x[x < 0]) if len(x[x < 0]) > 0 else None)
    , PRV_FRST_DUE_MAX = ("DAYS_FIRST_DUE", lambda x: max(x[x < 0]) if len(x[x < 0]) > 0 else None)
    , PRV_LST1_DUE_MIN = ("DAYS_LAST_DUE_1ST_VERSION", lambda x:min(x[x < 365243.00]) if len(x[x < 365243.00]) > 0 else None)
    , PRV_LST1_DUE_MAX = ("DAYS_LAST_DUE_1ST_VERSION", lambda x:max(x[x < 365243.00]) if len(x[x < 365243.00]) > 0 else None)
    , PRV_LST_DUE_MIN = ("DAYS_LAST_DUE", lambda x: min(x[x < 365243.00]) if len(x[x < 365243.00]) > 0 else None)
    , PRV_LST_DUE_MAX = ("DAYS_LAST_DUE", lambda  x:max(x[x < 365243.00]) if len(x[x < 365243.00]) > 0 else None)
    , PRV_DAYS_TER_MIN = ("DAYS_TERMINATION", lambda x: min(x[x < 365243.00]) if len(x[x < 365243.00]) > 0 else None)
    , PRV_DAYS_TER_MAX = ("DAYS_TERMINATION", lambda x:max(x[x < 365243.00]) if len(x[x < 365243.00]) > 0 else None)
    , PRV_INS_FLG_CNT = ("NFLAG_INSURED_ON_APPROVAL", "sum")
)

Summary_data.shape

(338857, 53)

In [23]:
Summary_data[Summary_data['SK_ID_CURR']==108129]

,SK_ID_CURR,PRV_CNT,PRV_CL_CNT,PRV_CONL_CNT,PRV_RL_CNT,PRV_ANNU_MAX,PRV_ANNU_MIN,PRV_AMT_APP_MAX,PRV_AMT_APP_MIN,PRV_AMT_CRD_MAX,...,PRV_DAYS_DRWN_MAX,PRV_FRST_DUE_MIN,PRV_FRST_DUE_MAX,PRV_LST1_DUE_MIN,PRV_LST1_DUE_MAX,PRV_LST_DUE_MIN,PRV_LST_DUE_MAX,PRV_DAYS_TER_MIN,PRV_DAYS_TER_MAX,PRV_INS_FLG_CNT
7769,108129,6,2,3,1,"25,188.62","4,830.93","607,500.00","33,052.50","679,671.00",...,-713.00,"-1,176.00",-134.00,-846.00,916.00,-846.00,-155.00,-840.00,-61.00,2.00


In [24]:
combined_data=app_data[['SK_ID_CURR','TARGET']].merge(Summary_data,on='SK_ID_CURR',how='left',suffixes=('_1','_2'))
combined_data.shape

(307511, 54)

In [25]:
# correlation with Target - target value
corr_features = pd.DataFrame()
corr_features['FI'] = combined_data.corr()['TARGET']
corr_features['FI_MOD']  = corr_features['FI'].abs()
print(corr_features.sort_values(by='FI_MOD',ascending=False).head(50)) 
#corr_features.style.background_gradient(cmap='viridis')

                     FI  FI_MOD
TARGET             1.00    1.00
PRV_DAYS_DRWN_MAX  0.10    0.10
PRV_DAYS_DRWN_MIN  0.10    0.10
PRV_RFS_CNT        0.06    0.06
PRV_REJ_RSN        0.06    0.06
PRV_PRD_TYP        0.06    0.06
PRV_FRST_DUE_MIN   0.05    0.05
PRV_DAYS_DES_MIN   0.05    0.05
PRV_LST1_DUE_MIN   0.05    0.05
PRV_LST_DUE_MIN    0.05    0.05
PRV_DAYS_TER_MIN   0.05    0.05
PRV_RL_CNT         0.05    0.05
PRV_RT_DPAY_MAX   -0.04    0.04
PRV_YILD_TYP       0.04    0.04
PRV_CL_PUR         0.04    0.04
PRV_TYP_OTHR       0.04    0.04
PRV_ANNU_MIN      -0.03    0.03
PRV_SLR_TYP       -0.03    0.03
PRV_RT_INT_P_MAX   0.03    0.03
PRV_RT_INT_P_MIN   0.03    0.03
PRV_PYMT_MIN       0.03    0.03
PRV_PYMT_MAX       0.03    0.03
PRV_ANNU_MAX      -0.03    0.03
PRV_AMT_DPAY_MAX  -0.03    0.03
PRV_PRD_CSH        0.02    0.02
PRV_CL_CNT         0.02    0.02
PRV_CONL_CNT      -0.02    0.02
PRV_AMT_APP_MIN   -0.02    0.02
PRV_AMT_GPRC_MIN  -0.02    0.02
PRV_AMT_DPAY_MIN  -0.02    0.02
PRV_CNT 